In [8]:
import os
import pandas as pd
import numpy as np

In [9]:
# Function 1: Volatility Sorting
def volatility_sorting(df):
    # Rank stocks based on volatility_stock and create quintiles
    df['Volatility_Rank'] = df['volatility_stock'].rank(method='first')
    df['Volatility_Quintile'] = pd.qcut(df['Volatility_Rank'], 5, labels=[1, 2, 3, 4, 5])
    return df

In [10]:
# Function 2: Correlation Sorting and Classification
def correlation_sorting(df):
    # Initialize columns for correlation rank and HL_Correlation classification
    df['Correlation_Rank'] = np.nan
    df['HL_Correlation'] = np.nan
    
    # Process each volatility quintile separately
    for quintile in df['Volatility_Quintile'].unique():
        quintile_df = df[df['Volatility_Quintile'] == quintile]
        
        # Rank correlations within each quintile
        quintile_df['Correlation_Rank'] = quintile_df['3year_rolling_correlation'].rank(method='first')
        avg_z_q = quintile_df['Correlation_Rank'].mean()
        
        # Assign High/Low correlation based on avg_z_q
        df.loc[quintile_df.index, 'HL_Correlation'] = quintile_df.apply(
            lambda row: f"{quintile}_H" if row['Correlation_Rank'] > avg_z_q 
                        else (f"{quintile}_L" if row['Correlation_Rank'] < avg_z_q else np.nan),
            axis=1
        )
        
        # Update the DataFrame with new correlation ranks
        df.loc[quintile_df.index, 'Correlation_Rank'] = quintile_df['Correlation_Rank']
        
    return df

In [11]:
# Function 3: Weight Calculation
def calculate_weights(df):
    df['NormalizeConstant'] = np.nan
    df['Weight'] = np.nan
    
    # Process each HL_Correlation portfolio
    for hl_portfolio in df['HL_Correlation'].unique():
        if pd.isna(hl_portfolio):
            continue
        
        # Filter the subset of stocks in the current HL_Correlation portfolio
        hl_df = df[df['HL_Correlation'] == hl_portfolio]
        quintile = int(hl_portfolio.split('_')[0])
        
        # Calculate the average rank for the corresponding volatility quintile
        avg_z_q = df[df['Volatility_Quintile'] == quintile]['Correlation_Rank'].mean()
        
        # Calculate the normalizing constant k(q)
        k_q = (2 / np.sum(np.abs(hl_df['Correlation_Rank'] - avg_z_q)))/2
        
        # Assign k(q) to the NormalizeConstant column
        df.loc[hl_df.index, 'NormalizeConstant'] = k_q
        
        # Calculate weight W(q_H) or W(q_L) for each stock in the HL_Correlation portfolio
        df.loc[hl_df.index, 'Weight'] = (k_q * np.abs(hl_df['Correlation_Rank'] - avg_z_q))
    
    return df

In [12]:
# Function 4: Calculate Returns, Betas, and BAC
def calculate_returns_betas(df):
    # Calculate Weighted Returns
    df['Weighted_Return'] = df['Weight'] * df['ROI_Lag1']
    
    # Calculate Weighted Betas
    df['Weighted_Beta'] = df['Weight'] * df['Adjusted_Beta']
    
    results = {}
    
    # For each quintile and correlation type, calculate the sum of weighted return and beta
    for quintile in df['Volatility_Quintile'].unique():
        for corr_type in ['H', 'L']:
            hl_portfolio = f"{quintile}_{corr_type}"
            subset_df = df[df['HL_Correlation'] == hl_portfolio]
            
            if not subset_df.empty:
                r_key = f"R({corr_type},{quintile})"
                b_key = f"B({quintile},{corr_type})"
                
                results[r_key] = subset_df['Weighted_Return'].sum()
                results[b_key] = subset_df['Weighted_Beta'].sum()
    
    # Calculate BAC returns for each volatility quintile
    rf = df['RF_Lag1'].iloc[0]  # Use any row that has data in risk-free rate column
    for quintile in df['Volatility_Quintile'].unique():
        r_H = results.get(f"R(H,{quintile})", 0)
        r_L = results.get(f"R(L,{quintile})", 0)
        b_H = results.get(f"B({quintile},H)", 0)
        b_L = results.get(f"B({quintile},L)", 0)
        
        # Calculate r_BAC(q)
        if b_L != 0 and b_H != 0:
            r_bac_key = f"r_BAC({quintile})"
            results[r_bac_key] = (1 / b_L) * (r_L - rf) - (1 / b_H) * (r_H - rf)
    
    # Calculate overall r_BAC for the month
    bac_keys = [f"r_BAC({q})" for q in df['Volatility_Quintile'].unique() if f"r_BAC({q})" in results]
    results['r_BAC'] = np.mean([results[key] for key in bac_keys if key in results])
    
    return results

In [13]:
# Function 5: Create Summary Sheet and Save (Modified)
def create_summary_sheet(results, month_year, output_file_path):
    # Create DataFrame for summary results
    summary_df = pd.DataFrame(results, index=[0])
    summary_df['Month_Year'] = month_year
    summary_df = summary_df[['Month_Year'] + [col for col in summary_df.columns if col != 'Month_Year']]
    
    # Change the output file path to a .csv file
    csv_output_path = output_file_path.replace('.csv', '_summary.csv')
    
    # Save the summary DataFrame to a CSV file
    summary_df.to_csv(csv_output_path, index=False)

    print(f"Summary sheet saved for Month_Year: {month_year} at {csv_output_path}")

In [14]:
# Main function to process each portfolio file (Revised)
def process_portfolio_files(portfolio_folder, output_folder, output_folder2):
    # Loop over all CSV files in the folder
    for filename in os.listdir(portfolio_folder):
        if filename.startswith("PORT_") and filename.endswith(".csv"):
            # Construct the full file path
            file_path = os.path.join(portfolio_folder, filename)
            
            # Read the portfolio data
            df = pd.read_csv(file_path)
            
            # Extract month and year from filename (remove ".csv")
            base_filename = filename.replace(".csv", "")
            month_year = base_filename.replace("PORT_", "")
            
            # Apply volatility sorting
            df = volatility_sorting(df)
            
            # Apply correlation sorting and classification
            df = correlation_sorting(df)
            
            # Calculate weights
            df = calculate_weights(df)
            
            # Calculate returns, betas, and BAC
            results = calculate_returns_betas(df)
            
            # Save the complete result file after all transformations (functions 1 to 4)
            combined_result_path = os.path.join(output_folder, f"{base_filename}_results.csv")
            df.to_csv(combined_result_path, index=False)
            print(f"Intermediate results saved for {filename} at {combined_result_path}")
            
            # Construct output file path for the summary sheet
            summary_file_path = os.path.join(output_folder2, f"{base_filename}.csv")
            
            # Save the summary sheet
            create_summary_sheet(results, month_year, summary_file_path)
            print(f"Summary sheet saved for {filename} at {summary_file_path}")

# Example usage
portfolio_path = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\1_Input\06_PortFolio\05_Port_Month_Year_FullData"
output_path = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\BAC_DATA"
output_path2 = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC"



In [15]:
process_portfolio_files(portfolio_path, output_path, output_path2)


C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_df['Correlation_Rank'] = quintile_df['3year_rolling_correlation'].rank(method='first')
C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['1_H' '1_H' '1_L' '1_H' '1_H' '1_H' '1_L' '1_L' '1_L' '1_L' '1_H' '1_L'
 '1_H' '1_L' '1_H' '1_L']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[quintile_df.index, 'HL_Correlation'] = quintile_df.apply(
C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py

Intermediate results saved for PORT_2012_1.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\BAC_DATA\PORT_2012_1_results.csv
Summary sheet saved for Month_Year: 2012_1 at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2012_1_summary.csv
Summary sheet saved for PORT_2012_1.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2012_1.csv
Intermediate results saved for PORT_2012_10.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\BAC_DATA\PORT_2012_10_results.csv
Summary sheet saved for Month_Year: 2012_10 at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2012_10_summary.csv
Summary sheet saved for PORT_2012_10.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2012_10.csv
Intermediate results saved for PORT_2012_11.csv at C:\Users\Kantinant Sukkert\Desktop\BAC 

C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_df['Correlation_Rank'] = quintile_df['3year_rolling_correlation'].rank(method='first')
C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['2_L' '2_H' '2_H' '2_L' '2_L' '2_L' '2_L' '2_H' '2_H' '2_H' '2_H' '2_L'
 '2_H' '2_L' '2_L' '2_H']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[quintile_df.index, 'HL_Correlation'] = quintile_df.apply(
C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py

Intermediate results saved for PORT_2013_11.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\BAC_DATA\PORT_2013_11_results.csv
Summary sheet saved for Month_Year: 2013_11 at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2013_11_summary.csv
Summary sheet saved for PORT_2013_11.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2013_11.csv
Intermediate results saved for PORT_2013_12.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\BAC_DATA\PORT_2013_12_results.csv
Summary sheet saved for Month_Year: 2013_12 at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2013_12_summary.csv
Summary sheet saved for PORT_2013_12.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2013_12.csv
Intermediate results saved for PORT_2013_2.csv at C:\Users\Kantinant Sukkert\Desktop

C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_df['Correlation_Rank'] = quintile_df['3year_rolling_correlation'].rank(method='first')
C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['2_L' '2_H' '2_L' '2_L' '2_L' '2_L' '2_H' '2_H' '2_L' '2_H' '2_L' nan
 '2_L' '2_H' '2_H' '2_H' '2_H']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[quintile_df.index, 'HL_Correlation'] = quintile_df.apply(
C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\65753386

...

C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_df['Correlation_Rank'] = quintile_df['3year_rolling_correlation'].rank(method='first')
C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['4_H' '4_H' '4_H' '4_H' '4_L' '4_L' nan '4_H' '4_L' '4_L' '4_H' '4_L'
 '4_H' '4_H' '4_L' '4_L' '4_L']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[quintile_df.index, 'HL_Correlation'] = quintile_df.apply(
C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\65753386

Intermediate results saved for PORT_2023_3.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\BAC_DATA\PORT_2023_3_results.csv
Summary sheet saved for Month_Year: 2023_3 at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2023_3_summary.csv
Summary sheet saved for PORT_2023_3.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2023_3.csv
Intermediate results saved for PORT_2023_4.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\BAC_DATA\PORT_2023_4_results.csv
Summary sheet saved for Month_Year: 2023_4 at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2023_4_summary.csv
Summary sheet saved for PORT_2023_4.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2023_4.csv
Intermediate results saved for PORT_2023_5.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Themati

C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_df['Correlation_Rank'] = quintile_df['3year_rolling_correlation'].rank(method='first')
C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['3_H' nan '3_L' '3_L' '3_H' '3_L' '3_L' '3_H' '3_H' '3_H' '3_L' '3_L'
 '3_H' '3_L' '3_L' '3_H' '3_H']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[quintile_df.index, 'HL_Correlation'] = quintile_df.apply(
C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\65753386

Intermediate results saved for PORT_2023_8.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\BAC_DATA\PORT_2023_8_results.csv
Summary sheet saved for Month_Year: 2023_8 at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2023_8_summary.csv
Summary sheet saved for PORT_2023_8.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2023_8.csv
Intermediate results saved for PORT_2023_9.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\BAC_DATA\PORT_2023_9_results.csv
Summary sheet saved for Month_Year: 2023_9 at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2023_9_summary.csv
Summary sheet saved for PORT_2023_9.csv at C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\06_BAC_Summary\Summary_BAC\PORT_2023_9.csv


C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_df['Correlation_Rank'] = quintile_df['3year_rolling_correlation'].rank(method='first')
C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\657533868.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['3_L' '3_H' '3_H' '3_L' '3_L' '3_L' '3_H' '3_L' '3_L' '3_H' '3_H' '3_H'
 '3_H' '3_L' '3_L' '3_L' '3_H' '3_H']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[quintile_df.index, 'HL_Correlation'] = quintile_df.apply(
C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_8716\